# Fitbit Food Data


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src/')
import plotutils
import fitbitqueries as fbq

In [3]:
import fitbit
import os
import getpass
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## Connect to fitbit API

In [15]:
os.environ['client_id'] = '22BC6L'
os.environ['client_secret'] = 'e8348b3b97bba4db4fb142928cedfc31'

In [16]:
!/Users/hasannagib/opt/anaconda3/envs/fitbit/bin/python ../src/generate_tokens.py $client_id $client_secret

[25/Dec/2019:13:02:58] ENGINE Listening for SIGTERM.
[25/Dec/2019:13:02:58] ENGINE Listening for SIGHUP.
[25/Dec/2019:13:02:58] ENGINE Listening for SIGUSR1.
[25/Dec/2019:13:02:58] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[25/Dec/2019:13:02:58] ENGINE Started monitor thread 'Autoreloader'.
[25/Dec/2019:13:02:59] ENGINE Serving on http://127.0.0.1:8080
[25/Dec/2019:13:02:59] ENGINE Bus STARTED
127.0.0.1 - - [25/Dec/2019:13:03:00] "GET /?code=06b3e7d780f55c2d363adfa4e7a84e78b909181e&state=njwfy3A72NyYh2Q2atMhHM5tm0rnvt HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36"
[25/Dec/2019:13:03:01] ENGINE Bus STOPPING
[25/Dec/2019:13:03:01] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[25/Dec/2019:13:03:01] ENGINE Stopped thread 'Autoreloader'.
[25/Dec/2019:13:03:01] ENGINE Bus STOPPED
[25/Dec/2019:13:03:01] ENGIN

In [17]:
with open('../data/access_token.txt','r') as f:
    os.environ['fitbit_access_token'] = f.read()

with open('../data/refresh_token.txt','r') as f:
    os.environ['fitbit_refresh_token'] = f.read()
    
auth_client = fitbit.Fitbit(
    os.environ['client_id'], 
    os.environ['client_secret'],
    os.environ['fitbit_access_token'],
    os.environ['fitbit_refresh_token']
)

## Query food data

In [130]:
date_list = pd.date_range('2019-11-24', '2019-12-24')

food_logs = [auth_client.foods_log(date) for date in date_list]
macros_data = [[food_log['summary'], date] for food_log, date in zip(food_logs, date_list)]
food_data = []

for food_log in food_logs:
    for item in food_log['foods']:
        food_data.append([item['logDate'], item['loggedFood']['name'], item['loggedFood']['calories']])
    

In [152]:
# Prepare foods dataframes
df_foods = pd.DataFrame(food_data, columns=['date', 'name', 'calories'])
df_macros = pd.DataFrame([data[0] for data in macros_data])
df_macros['date'] = [data[1] for data in macros_data]
df_top_foods = df_foods.groupby('name').sum().sort_values('calories', ascending=False)
df_top_foods['% of total calories'] = 100*(df_top_foods/df_foods.sum()['calories'])
df_top_foods['% calories (cumulative)'] = df_top_foods['% of total calories'].cumsum()

## What do I actually eat...
Let's inspect the food logs from the last month. Where do most of my calories come from?

In [232]:
print(f"Food by cals:\n{df_foods['date'].min()} to {df_foods['date'].max()}")
df_top_foods.head(30)

Food by cals:
2019-11-24 to 2019-12-24


,calories,% of total calories,% calories (cumulative)
name,,,
"Recovery Formula, Chocolate",8438,8.810232,8.810232
12 Grain Bagels,6440,6.724093,15.534325
Walnuts,4885,5.100496,20.634821
Nuttin' Better (Regular),3929,4.102323,24.737144
Vegetarian Poke Bowl,3584,3.742104,28.479248
Avocados,3336,3.483164,31.962412
"Peanut Butter, Smooth",3240,3.382929,35.345341
Salmon Poke Bowl,3060,3.194988,38.540329
Pumpkin Seeds,2824,2.948577,41.488906


In [231]:
plotutils.top_food_plot(df_top_foods.head(50))